In [1]:
import pandas as pd
import numpy as np
import lightfm
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k, reciprocal_rank, auc_score
from lightfm.cross_validation import random_train_test_split
from dataloading import DataLoader
from datapreparation_py import DataPreparation
from lightFM_adoptor import LightFMAdaptor
from cv_py import CrossValidation
from implicit.evaluation import ranking_metrics_at_k, train_test_split
from scipy import sparse

In [2]:
%cd C:\Users\781110104\OneDrive - Genpact\Documents\VSCode

C:\Users\781110104\OneDrive - Genpact\Documents\VSCode


In [3]:
dl = DataLoader()
user_item_co_t = dl.import_data('TEREX', 'CO', 'df')

In [4]:
user_item_co_t_filtered = dl.remove_low_interact_items(user_item_co_t, 1)

In [5]:
user_item_filtered_log_t = dl.log_scale_df(user_item_co_t_filtered, 0.01)

In [6]:
dp = DataPreparation(user_item_filtered_log_t)

In [7]:
interactions_csr, weights_csr = dp.get_interaction_data()

In [28]:
from scipy import sparse
sparse.save_npz('interactions_csr_t.npz', interactions_csr)
sparse.save_npz('weights_csr_t.npz', weights_csr)

In [2]:
interactions_csr = sparse.load_npz('interactions_csr_t.npz')
weights_csr = sparse.load_npz('weights_csr_t.npz')

In [16]:
user_features = ['country', 'brand', 'currency']
item_features = ['group2', 'movement_code', 'cost_class']
feature_input = dp.get_feature_data(OEM='TEREX', user_features=user_features, item_features=item_features)

In [17]:
userf, itemf = feature_input

In [18]:
user_features = ['country']
item_features = ['group2']
userf_country, itemf_group2 = dp.get_feature_data(OEM='TEREX', user_features=user_features, item_features=item_features)

In [19]:
user_features = ['brand']
item_features = ['movement_code']
userf_brand, itemf_move = dp.get_feature_data(OEM='TEREX', user_features=user_features, item_features=item_features)

In [20]:
user_features = ['currency']
item_features = ['cost_class']
userf_curr, itemf_cost = dp.get_feature_data(OEM='TEREX', user_features=user_features, item_features=item_features)

In [9]:
user_features = ['country', 'brand']
item_features = ['group2', 'movement_code']
userf_country_brand, itemf_group2_move = dp.get_feature_data(OEM='TEREX', user_features=user_features, item_features=item_features)

In [21]:
user_features = ['country', 'currency']
item_features = ['group2', 'cost_class']
userf_country_curr, itemf_group2_cost = dp.get_feature_data(OEM='TEREX', user_features=user_features, item_features=item_features)

In [22]:
user_features = ['brand', 'currency']
item_features = ['movement_code', 'cost_class']
userf_brand_curr, itemf_move_cost = dp.get_feature_data(OEM='TEREX', user_features=user_features, item_features=item_features)

In [29]:
sparse.save_npz('itemf_move_cost_t.npz', itemf_move_cost[0])
sparse.save_npz('itemf_group_cost_t.npz', itemf_group2_cost[0])
sparse.save_npz('itemf_group_move_t.npz', itemf_group2_move[0])
sparse.save_npz('itemf_cost_t.npz', itemf_cost[0])
sparse.save_npz('itemf_move_t.npz', itemf_move[0])
sparse.save_npz('itemf_group_t.npz', itemf_group2[0])
sparse.save_npz('itemf_t.npz', itemf)

In [3]:
itemf_move_cost_t = sparse.load_npz('itemf_move_cost_t.npz')
itemf_group_cost_t = sparse.load_npz('itemf_group_cost_t.npz')
itemf_group_move_t = sparse.load_npz('itemf_group_move_t.npz')
itemf_cost_t = sparse.load_npz('itemf_cost_t.npz')
itemf_move_t = sparse.load_npz('itemf_move_t.npz')
itemf_group_t = sparse.load_npz('itemf_group_t.npz')
itemf_t = sparse.load_npz('itemf_t.npz')

In [31]:
sparse.save_npz('userf_t.npz', userf)
sparse.save_npz('userf_country_t.npz', userf_country[0])
sparse.save_npz('userf_curr_t.npz', userf_curr[0])
sparse.save_npz('userf_brand_t.npz', userf_brand[0])
sparse.save_npz('userf_country_curr_t.npz', userf_country_curr[0])
sparse.save_npz('userf_country_brand_t.npz', userf_country_brand[0])
sparse.save_npz('userf_brand_curr_t.npz', userf_brand_curr[0])

In [4]:
userf_t = sparse.load_npz('userf_t.npz')
userf_country_t = sparse.load_npz('userf_country_t.npz')
userf_curr_t = sparse.load_npz('userf_curr_t.npz')
userf_brand_t = sparse.load_npz('userf_brand_t.npz')
userf_country_curr_t = sparse.load_npz('userf_country_curr_t.npz')
userf_country_brand_t = sparse.load_npz('userf_country_brand_t.npz')
userf_brand_curr_t = sparse.load_npz('userf_brand_curr_t.npz')

In [5]:
train_i, test_i = train_test_split(interactions_csr, 0.8, 22)
train_w, test_w = train_test_split(weights_csr, 0.8, 22)

In [6]:
cal_w, val_w = train_test_split(train_w, 0.8, 22)
cal_i, val_i = train_test_split(train_i, 0.8, 22)

In [7]:
cv = CrossValidation(5)

tuning empty model

In [14]:
space_FM_warp = {'factors' : [64], 'regularization' : [0.00095, 0.001, 0.0015], 'learning_rate' : [0.00095, 0.001, 0.0015], 'iterations' : [15], 'loss' : ['warp'], 'max_sampled' : [10, 15, 30]}

uf_combi = [None]
if_combi = [None]

uf_names = ['None']
if_names = ['None']

In [15]:
results_warp = cv.tune_FM(space=space_FM_warp, user_f=uf_combi, item_f=if_combi, uf_names=uf_names, 
if_names=if_names, train=cal_w, test=val_w, exclude=test_w)

In [16]:
results_warp.sort_values('precision', ascending=False).head()

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr,uf_name,if_name
13,64,0.00100,0.001,15,warp,15,0.4000,0.257977,0.413110,0.510754,0.268572,None,None
14,64,0.00100,0.001,15,warp,30,0.4000,0.257977,0.413110,0.510754,0.268572,None,None
12,64,0.00100,0.001,15,warp,10,0.4000,0.257977,0.413110,0.510754,0.268579,None,None
3,64,0.00095,0.001,15,warp,10,0.3975,0.257264,0.413993,0.510736,0.268854,None,None
4,64,0.00095,0.001,15,warp,15,0.3975,0.257264,0.413993,0.510736,0.268848,None,None


Testing the model with simple parameter tuning:

First step: fit empty and full models with few parameter combinations. Objective: get a decent parameter combination, that will fit all models more or less!

Second step: fit model with all possible feature combinations to the parameters, that came out best in step 1:

In [28]:
space_FM_warp = {'factors' : [64], 'regularization' : [0.001], 'learning_rate' : [0.001], 'iterations' : [15], 'loss' : ['warp'], 'max_sampled' : [30]}

uf_combi = [None, userf_country_t, userf_brand_t, userf_curr_t, userf_country_brand_t, userf_country_curr_t, userf_brand_curr_t, userf_t]
if_combi = [None, itemf_group_t, itemf_move_t, itemf_cost_t, itemf_group_move_t, itemf_group_cost_t, itemf_move_cost_t, itemf_t]

uf_names = ['None', 'userf_country_t', 'userf_brand_t', 'userf_curr_t', 'userf_country_brand_t', 'userf_country_curr_t', 'userf_brand_curr_t', 'userf_t']
if_names = ['None', 'itemf_group_t', 'itemf_move_t', 'itemf_cost_t', 'itemf_group_move_t', 'itemf_group_cost_t', 'itemf_move_cost_t', 'itemf_t']

In [29]:
results_warp = cv.tune_FM(space=space_FM_warp, user_f=uf_combi, item_f=if_combi, uf_names=uf_names, 
if_names=if_names, train=cal_w, test=val_w, exclude=test_w)

In [30]:
results_warp.sort_values('precision', ascending=False).head(10)

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr,uf_name,if_name
0,64,0.001,0.001,15,warp,30,0.4250,0.297576,0.464037,0.512037,0.135649,userf_t,None
0,64,0.001,0.001,15,warp,30,0.4025,0.281807,0.441873,0.510504,0.143439,userf_country_brand_t,None
0,64,0.001,0.001,15,warp,30,0.4025,0.270755,0.432880,0.511253,0.137911,userf_brand_curr_t,None
0,64,0.001,0.001,15,warp,30,0.4000,0.262103,0.422051,0.510650,0.138477,userf_brand_t,None
0,64,0.001,0.001,15,warp,30,0.4000,0.257977,0.413110,0.510754,0.268572,None,None
0,64,0.001,0.001,15,warp,30,0.3250,0.199091,0.340000,0.508808,0.183874,userf_country_t,None
0,64,0.001,0.001,15,warp,30,0.3050,0.186348,0.326103,0.509829,0.216172,userf_country_curr_t,None
0,64,0.001,0.001,15,warp,30,0.2975,0.187115,0.323048,0.509990,0.222997,userf_curr_t,None
0,64,0.001,0.001,15,warp,30,0.2575,0.160125,0.287021,0.506605,0.253334,None,itemf_group_t
0,64,0.001,0.001,15,warp,30,0.2050,0.106122,0.206717,0.504993,0.217395,userf_t,itemf_t


Step 3: take feature combination with best results of step 2 and tune parameters:

In [11]:
space_FM_warp = {'factors' : [64], 'regularization' : [0.001, 0.0015, 0.002], 'learning_rate' : [0.001, 0.0015, 0.002], 'iterations' : [15], 'loss' : ['warp'], 'max_sampled' : [5, 10, 15]}

uf_combi = [userf_t]
if_combi = [None]

uf_names = ['userf_t']
if_names = ['None']

In [12]:
results_warp = cv.tune_FM(space=space_FM_warp, user_f=uf_combi, item_f=if_combi, uf_names=uf_names, 
if_names=if_names, train=cal_w, test=val_w, exclude=test_w)

In [13]:
results_warp.sort_values('precision', ascending=False).head(10)

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr,uf_name,if_name
13,64,0.0015,0.0015,15,warp,10,0.4650,0.329563,0.489979,0.512652,0.129341,userf_t,None
20,64,0.0020,0.0010,15,warp,15,0.4575,0.311167,0.477246,0.512509,0.135596,userf_t,None
0,64,0.0010,0.0010,15,warp,5,0.4500,0.320114,0.485222,0.512487,0.135072,userf_t,None
11,64,0.0015,0.0010,15,warp,15,0.4475,0.309592,0.477402,0.512263,0.135781,userf_t,None
19,64,0.0020,0.0010,15,warp,10,0.4450,0.308976,0.476065,0.512380,0.135711,userf_t,None
18,64,0.0020,0.0010,15,warp,5,0.4450,0.305739,0.465849,0.512293,0.134817,userf_t,None
25,64,0.0020,0.0020,15,warp,10,0.4425,0.293427,0.454159,0.511822,0.123978,userf_t,None
9,64,0.0015,0.0010,15,warp,5,0.4425,0.302211,0.468645,0.512442,0.134640,userf_t,None
10,64,0.0015,0.0010,15,warp,10,0.4425,0.306811,0.472405,0.512336,0.135794,userf_t,None
23,64,0.0020,0.0015,15,warp,15,0.4375,0.289292,0.457142,0.512122,0.129437,userf_t,None


Frage: Warum sind die Ergebnisse bei widerholtem fitten mit gleichen parametern unterschiedlich? Vlt prüfen mit purem modell LightFM, ob es da auch schwankt?

In [36]:
space_FM_warp = {'factors' : [64], 'regularization' : [0.0011, 0.0013, 0.0015], 'learning_rate' : [0.008, 0.01, 0.012], 'iterations' : [15], 'loss' : ['warp'], 'max_sampled' : [25, 30, 35]}

uf_combi = [None]
if_combi = [itemf_t]

uf_names = ['None']
if_names = ['itemf_t']

In [37]:
results_warp = cv.tune_FM(space=space_FM_warp, user_f=uf_combi, item_f=if_combi, uf_names=uf_names, 
if_names=if_names, train=cal_w, test=val_w, exclude=test_w)

In [38]:
results_warp.sort_values('precision', ascending=False).head()

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr,uf_name,if_name
13,64,0.0013,0.010,15,warp,30,0.4225,0.285694,0.432347,0.512191,0.111061,None,itemf_t
3,64,0.0011,0.010,15,warp,25,0.4175,0.271599,0.424959,0.511376,0.110586,None,itemf_t
22,64,0.0015,0.010,15,warp,30,0.4150,0.276514,0.434926,0.512075,0.111242,None,itemf_t
14,64,0.0013,0.010,15,warp,35,0.4075,0.260532,0.412561,0.511281,0.111437,None,itemf_t
11,64,0.0013,0.008,15,warp,35,0.4050,0.276652,0.427319,0.511547,0.117799,None,itemf_t


In [60]:
space_FM_warp = {'factors' : [64], 'regularization' : [0.00003, 0.00005, 0.00007], 'learning_rate' : [0.006, 0.008, 0.01], 'iterations' : [15], 'loss' : ['warp'], 'max_sampled' : [65, 70, 75]}

uf_combi = [userf_t]
if_combi = [itemf_t]

uf_names = ['userf_t']
if_names = ['itemf_t']

In [61]:
results_warp = cv.tune_FM(space=space_FM_warp, user_f=uf_combi, item_f=if_combi, uf_names=uf_names, 
if_names=if_names, train=cal_w, test=val_w, exclude=test_w)

In [62]:
results_warp.sort_values('precision', ascending=False).head()

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr,uf_name,if_name
13,64,0.00005,0.008,15,warp,70,0.4050,0.276784,0.431078,0.511414,0.115154,userf_t,itemf_t
21,64,0.00007,0.008,15,warp,65,0.4025,0.253734,0.421104,0.511536,0.115216,userf_t,itemf_t
4,64,0.00003,0.008,15,warp,70,0.4000,0.255372,0.408059,0.511331,0.115220,userf_t,itemf_t
15,64,0.00005,0.010,15,warp,65,0.3950,0.277036,0.439075,0.511641,0.117835,userf_t,itemf_t
20,64,0.00007,0.006,15,warp,75,0.3925,0.275483,0.425232,0.510521,0.115592,userf_t,itemf_t


Now user features individually:

In [66]:
space_FM_warp = {'factors' : [64], 'regularization' : [0.0005, 0.001, 0.0015], 'learning_rate' : [0.0015, 0.002, 0.0025], 'iterations' : [15], 'loss' : ['warp'], 'max_sampled' : [10, 15, 20]}

uf_combi = [userf_country_t]
if_combi = [None]

uf_names = ['userf_country_t']
if_names = ['None']

In [67]:
results_warp = cv.tune_FM(space=space_FM_warp, user_f=uf_combi, item_f=if_combi, uf_names=uf_names, 
if_names=if_names, train=cal_w, test=val_w, exclude=test_w)

In [68]:
results_warp.sort_values('precision', ascending=False).head()

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr,uf_name,if_name
13,64,0.0010,0.0020,15,warp,15,0.3725,0.230451,0.386429,0.510603,0.159986,userf_country_t,None
4,64,0.0005,0.0020,15,warp,15,0.3650,0.227373,0.377122,0.509312,0.158923,userf_country_t,None
21,64,0.0015,0.0020,15,warp,10,0.3650,0.242125,0.393574,0.509423,0.160308,userf_country_t,None
15,64,0.0010,0.0025,15,warp,10,0.3625,0.229480,0.381227,0.509111,0.147849,userf_country_t,None
12,64,0.0010,0.0020,15,warp,10,0.3625,0.234771,0.390732,0.509249,0.158319,userf_country_t,None


In [72]:
space_FM_warp = {'factors' : [64], 'regularization' : [0.001, 0.0015, 0.002], 'learning_rate' : [0.0015, 0.002, 0.0025], 'iterations' : [15], 'loss' : ['warp'], 'max_sampled' : [15, 20, 25]}

uf_combi = [userf_brand_t]
if_combi = [None]

uf_names = ['userf_brand_t']
if_names = ['None']

In [73]:
results_warp = cv.tune_FM(space=space_FM_warp, user_f=uf_combi, item_f=if_combi, uf_names=uf_names, 
if_names=if_names, train=cal_w, test=val_w, exclude=test_w)

In [74]:
results_warp.sort_values('precision', ascending=False).head()

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr,uf_name,if_name
13,64,0.0015,0.0020,15,warp,20,0.4250,0.263096,0.415073,0.511219,0.134234,userf_brand_t,None
19,64,0.0020,0.0015,15,warp,20,0.4200,0.285790,0.440116,0.510999,0.139015,userf_brand_t,None
21,64,0.0020,0.0020,15,warp,15,0.4100,0.276094,0.423150,0.510109,0.134607,userf_brand_t,None
14,64,0.0015,0.0020,15,warp,25,0.4100,0.265256,0.414074,0.510705,0.133752,userf_brand_t,None
10,64,0.0015,0.0015,15,warp,20,0.4075,0.262318,0.418758,0.510562,0.138464,userf_brand_t,None


In [78]:
space_FM_warp = {'factors' : [64], 'regularization' : [0.0015, 0.002, 0.0025], 'learning_rate' : [0.002, 0.0025, 0.003], 'iterations' : [15], 'loss' : ['warp'], 'max_sampled' : [15, 20, 25]}

uf_combi = [userf_curr_t]
if_combi = [None]

uf_names = ['userf_curr_t']
if_names = ['None']

In [79]:
results_warp = cv.tune_FM(space=space_FM_warp, user_f=uf_combi, item_f=if_combi, uf_names=uf_names, 
if_names=if_names, train=cal_w, test=val_w, exclude=test_w)

In [80]:
results_warp.sort_values('precision', ascending=False).head()

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr,uf_name,if_name
13,64,0.0020,0.0025,15,warp,20,0.3275,0.198237,0.336499,0.510192,0.208179,userf_curr_t,None
4,64,0.0015,0.0025,15,warp,20,0.3250,0.207251,0.347004,0.510575,0.208665,userf_curr_t,None
8,64,0.0015,0.0030,15,warp,25,0.3225,0.209534,0.351676,0.510325,0.202513,userf_curr_t,None
12,64,0.0020,0.0025,15,warp,15,0.3200,0.190555,0.319977,0.510554,0.207444,userf_curr_t,None
5,64,0.0015,0.0025,15,warp,25,0.3150,0.193403,0.330668,0.510300,0.207902,userf_curr_t,None


In [83]:
space_FM_bpr_add = {'factors' : [40], 'regularization' : [0.0005], 
'learning_rate' : [0.01, 0.05, 0.1], 'iterations' : [10], 'loss' : ['bpr'], 'max_sampled' : [1]}

uf_combi = [None, userf_country, userf_brand, userf_curr, userf_country_brand, userf_country_curr, userf_brand_curr, userf]
if_combi = [None, itemf_group2, itemf_move, itemf_cost, itemf_group2_move, itemf_group2_cost, itemf_move_cost, itemf]

uf_names = ['None', 'userf_country', 'userf_brand', 'userf_curr', 'userf_country_brand', 'userf_country_curr', 'userf_brand_curr', 'userf']
if_names = ['None', 'itemf_group2', 'itemf_move', 'itemf_cost', 'itemf_group2_move', 'itemf_group2_cost', 'itemf_move_cost', 'itemf']

In [84]:
results_bpr_add = cv.tune_FM(space=space_FM_bpr_add, user_f=uf_combi, item_f=if_combi, uf_names=uf_names, if_names=if_names, train=cal_w, 
test=val_w, exclude=test_w)

In [85]:
results_bpr = pd.concat([results_bpr, results_bpr_add], axis=0)

In [87]:
results_bpr.sort_values('mpr', ascending=True).head(10)

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr,uf_name,if_name
1,40,0.0010,0.05,10,bpr,1,0.2550,0.141409,0.273022,0.507202,0.104139,None,itemf
1,40,0.0005,0.05,10,bpr,1,0.2700,0.156089,0.293850,0.507304,0.104282,None,itemf
1,40,0.0005,0.05,10,bpr,1,0.2600,0.136159,0.265025,0.506721,0.104620,None,itemf_group2_move
1,40,0.0010,0.05,10,bpr,1,0.2725,0.134807,0.269027,0.507060,0.105132,None,itemf_group2_move
1,40,0.0005,0.05,10,bpr,1,0.2475,0.130434,0.265882,0.506915,0.106064,None,itemf_group2_cost
1,40,0.0005,0.05,10,bpr,1,0.2725,0.143808,0.282722,0.507257,0.106255,None,itemf_group2
1,40,0.0010,0.05,10,bpr,1,0.2650,0.146232,0.281878,0.507409,0.107148,None,itemf_group2_cost
1,40,0.0010,0.05,10,bpr,1,0.3075,0.173261,0.319161,0.507898,0.108988,None,itemf_group2
0,40,0.0005,0.01,10,bpr,1,0.2375,0.124910,0.247752,0.505598,0.109298,userf,itemf
0,40,0.0010,0.01,10,bpr,1,0.2400,0.124324,0.249034,0.505683,0.109473,userf,itemf


In [47]:
results_df_tuned_warp.to_csv('fm_tuning_warp_t.csv')

In [4]:
#results_df_tuned = pd.read_csv('fm_tuning_t.csv')